In [4]:
import sympy
import sympy.simplify

from sympy import Integral, integrate

sympy.init_printing()
sympy.init_session()

IPython console for SymPy 1.13.2 (Python 3.11.10-64-bit) (ground types: gmpy)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.13.2/



In [6]:
T = sympy.Symbol('T')
DT = sympy.Symbol('DT')
S = sympy.Symbol('S')

In [14]:
t = sympy.Symbol('t')


interm1 = integrate(-S*t*t, (t, 0, DT))

interm2 = integrate(-S*DT*(t+DT), (t, 0, T))

interm3 = integrate(S*(t-DT)*(t + T + DT), (t, 0, 2*DT))

interm4 = integrate(S*DT*(t + T + 3*DT), (t, 0, T))

interm5 = integrate((S*(DT - t)*(t + 2*T + 3*DT)), (t, 0, DT))


sum_tot = interm1 + interm2 + interm3 + interm4 + interm5

sum_tot = sympy.simplify(sympy.expand(sum_tot))

print(sum_tot)

DT*S*(2*DT**2 + 3*DT*T + T**2)
